# Input Test Value in Functional Programing with Python

## get_value: basic function to retrive input value

In [2]:

def get_value(message):
    '''
    Return the value provided through input()
    
        Parameters:
            message (str): A message to display for the user
            
        Return:
            _ (str): the value provided by user
    
    '''
    return input(f'{message} :')

## The functions test: test the input value

In [7]:


def is_digit(message='Please provide a int'):
    '''
    Test if a given value can be converted to an integer object.
    
    If value non convertible, raise ValueError with personalised message.
    By default the Error message is 'Please provide a int'
    
    If value is convertible, return the given value. 
    
        Parameters:
            message (str): A personalised message if given value can't be convertible to interger object. 
            num (str): A given string value
            
        Returns:
            int_num (int): an interger object
    '''
    def init_message(num):
        try:
            int_num = int(num)
        except ValueError:
            raise ValueError(message)
        else: 
            return int(int_num)
        
    return init_message
    
      
def is_zero(num):
    try:
        7/int(num)
    except ZeroDivisionError:
        raise ZeroDivisionError('Please avoid 0 for denominator')
        
        
def is_dummy(num):
    return num


def test_value(func, recall, value):
    try:
        tested_value=func(value)
    except Exception as exp:
        print('error', exp)
        return recall()
    else:
        return value
    

In [8]:
type(is_digit()('1'))

int

In [ ]:
def get_value_and_test(*funcs, message):
    
    # first level - encapsulate the get get_value and test
    
    def launch_get_value():
        
        # second level - run de get value 
        
        # If one of the test function raises exception - recursion on this level 'launch_get_value' (see function test_value)
        
        *tests, get_value=funcs # destructure functions, given that the last one must be get_value
        
        value = get_value(message) # run get value and assigne the input to variable value
        
        def launch_test(*tests, value):  
            
            # third level - Recursive function
        
            len_tests = len(tests) 
            
            if len_tests ==1: # last test : end of the recursion 
                
                last_test = tests[0]
                
                tested_value_approved =  test_value(last_test, launch_get_value, value) # if last test raises exeception call launch_get_value
                
                return tested_value_approved # end of get_value_and_test return approved tested_value 
                
            else:
            
                *last_tests,  first_test = tests # get the last test of the list of test. If hierachy between tests (
                                                 # ie. input must resturn an int(str), this test would have to be picked first)
            
                tested_value = test_value(first_test, launch_get_value, value) # if last test raises exeception call launch_get_value

                # return launch_test(last_tests, value=tested_value) if len(last_tests)==1 else launch_test(*last_tests, value=tested_value)
                
                return launch_test(*last_tests, value=tested_value) # recursion of launch_test function wiht -1 function in test function

              
        return launch_test(*tests, value=value) # !!! launch_test FIRST RUN HERE 

    return launch_get_value
  